In [ ]:
#!git clone -b master https://github.com/charles9n/bert-sklearn
#!cd bert-sklearn; pip install .

In [1]:
import os
import math
import random
import csv
import sys

import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
import statistics as stats

from bert_sklearn import BertClassifier
from bert_sklearn import BertRegressor
from bert_sklearn import BertTokenClassifier
from bert_sklearn import load_model


In [2]:
import sys
sys.path.insert(0,"../src")

In [14]:
import vectorize
import helpers
import transformers
from utils import *
from loss.loss import *

from bert_sklearn_transformer import BertTransformer

In [4]:
full_df = pd.read_pickle("../data/full_df.pkl")

In [5]:
MAX_VOCAB = None
MAX_SEQ_LENGTH = 5000
full_df['TEXT_PROCESSED'] = vectorize.clean_notes(full_df, 'TEXT')

In [6]:
full_df['ICD9_GRP_LIST'] = full_df.ICD9_GRP.apply(lambda x: re.split(" +", x.strip()))

# Pipeline Training

In [ ]:
from transformers import BertConfig, BertForSequenceClassification, BertModel, BertTokenizer
from sklearn import svm
from sklearn.pipeline import Pipeline


In [11]:
# Another way of splitting the data

split = np.random.choice(
    ["train", "val", "test"],
    size=full_df.shape[0],
    p=[.7, .15, .15]
)
full_df["split"] = split

In [12]:
sample_df = full_df.sample(frac=0.3)

In [13]:
X_train = sample_df[sample_df["split"] == "train"]['TEXT']
y_train = mlb.fit_transform(sample_df.ICD9_GRP_LIST)


In [16]:
config = BertConfig.from_pretrained('bert-base-uncased') 

bert_tok = BertTokenizer(config)
bert_model = BertModel(config)

bert_transformer = BertTransformer(bert_tok, bert_model)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
classifier = MultinomialNB(fit_prior=True, class_prior = None)
classifier = LogisticRegression(solver='sag')
model = Pipeline(
    [
        ("vectorizer", bert_transformer),
        ("classifier", OneVsRestClassifier(classifier)),
    ]
)
model.fit(X_train, y_train)

# Training

In [ ]:
classifier = svm.LinearSVC(C=1.0, class_weight="balanced")
model = Pipeline(
    [
        ("vectorizer", bert_transformer),
        ("classifier", classifier),
    ]
)
model.fit(X_train, y_train)

# Adding TF-IDF

In [ ]:
from sklearn.feature_extraction.text import (
   CountVectorizer, TfidfTransformer
)

tf_idf = Pipeline([
    ("vect", CountVectorizer()),
    ("tfidf", TfidfTransformer())
    ])

model = Pipeline([
    ("union", FeatureUnion(transformer_list=[
        ("bert", bert_transformer),
        ("tf_idf", tf_idf)
        ])),
        ("classifier", classifier),
    ])

In [ ]:
model.fit(X_train, y_train)

In [8]:
# Not working 


# from sklearn.preprocessing import MultiLabelBinarizer
# mlb = MultiLabelBinarizer()

# X_train = sample_df['TEXT']
# y_train = mlb.fit_transform(sample_df.ICD9_GRP_LIST)
# model = BertClassifier(max_seq_length=128, train_batch_size=4)
# model_gradient_accumulation_steps = 4

# model
# model = model.fit(X_train, y_train)